# <div align="center"> 调研CV结果可视化 </div>

## kaggle

- [keras: flower-recognition-cnn-keras](https://www.kaggle.com/rajmehra03/flower-recognition-cnn-keras)

- [keras: keras-cnn-dog-or-cat-classification](https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification)

- [keras: introduction-to-cnn-keras](https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6#3.-CNN)


## tensorboard

- [pytorch](https://pytorch.org/docs/stable/tensorboard.html)

- [kaggle](https://www.kaggle.com/mlagunas/naive-unet-with-pytorch-tensorboard-logging)


### other

- [PSNR-SSIM(GPU上的相似度检测)](https://cvnote.ddlee.cn/2019/09/12/PSNR-SSIM-Python.html)

## Youtube

- [Tensorboard: torch](https://www.youtube.com/watch?v=pSexXMdruFM)

## Metrics

### 过程类

1. [x] 训练过程中的输入到模型的图片(增强变换之后)
2. [x] loss & loss_avg: training loss, Validation loss
3. [x] lr
4. [x] weight, bias and grad histogram


### 结果类

1. [x] accuracy & accuracy_topX
2. [x] confusion matrix
3. [x] model graph (tensorboard)
4. [ ] 展示分错次数多的分类
5. [ ] psnr & ssim
6. [x] 精确度, 召回率, F1, 
7. [ ] ROC曲线, AUC, 特异性


### 性能类

1. [x] cpu/gpu内存 利用率
2. [x] 训练速度

In [1]:
%reload_ext autoreload
%reload_ext tensorboard
%autoreload 2

In [2]:
import torch
import torchvision
import numpy as np
import cv2
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from k12libs.utils.nb_easy import K12AI_HOST_ADDR, K12AI_NBDATA_ROOT
from k12libs.utils.nb_easy import k12ai_set_notebook

In [3]:
k12ai_set_notebook(cellw=80)

In [4]:
tb_port = 6001
log_dir = f'{K12AI_NBDATA_ROOT}/tmp/mnist/logs_{tb_port}'
writer = SummaryWriter(log_dir=log_dir)

In [19]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST(f'{K12AI_NBDATA_ROOT}/datasets', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
model = torchvision.models.resnet50(False)
# Have ResNet model take in grayscale rather than RGB
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
images, labels = next(iter(trainloader))

grid = torchvision.utils.make_grid(images)
writer.add_image('images', grid, 0)
writer.add_graph(model, images)

<class 'torchvision.models.resnet.ResNet'>


In [6]:
for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [7]:
r = 5
for i in range(100):
    writer.add_scalars('run_14h', {'xsinx':i*np.sin(i/r),
                                    'xcosx':i*np.cos(i/r),
                                    'tanx': np.tan(i/r)}, i)

In [8]:
for i in range(10):
    x = np.random.random(1000)
    writer.add_histogram('distribution centers', x + i, i)

In [9]:
img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

img_HWC = np.zeros((100, 100, 3))
img_HWC[:, :, 0] = np.arange(0, 10000).reshape(100, 100) / 10000
img_HWC[:, :, 1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

writer.add_image('my_image', img, 0)

writer.add_image('my_image_HWC', img_HWC, 0, dataformats='HWC')

In [10]:
img_batch = np.zeros((16, 3, 100, 100))
for i in range(16):
    img_batch[i, 0] = np.arange(0, 10000).reshape(100, 100) / 10000 / 16 * i
    img_batch[i, 1] = (1 - np.arange(0, 10000).reshape(100, 100) / 10000) / 16 * i
    
writer.add_images('my_image_batch', img_batch, 0)

In [11]:
writer.add_text('lstm', 'This is an lstm', 0)
writer.add_text('rnn', 'This is an rnn', 10)

In [12]:
vertices_tensor = torch.as_tensor([
    [1, 1, 1],
    [-1, -1, 1],
    [1, -1, -1],
    [-1, 1, -1],
], dtype=torch.float).unsqueeze(0)
colors_tensor = torch.as_tensor([
    [255, 0, 0],
    [0, 255, 0],
    [0, 0, 255],
    [255, 0, 255],
], dtype=torch.int).unsqueeze(0)
faces_tensor = torch.as_tensor([
    [0, 2, 3],
    [0, 3, 1],
    [0, 1, 2],
    [1, 3, 2],
], dtype=torch.int).unsqueeze(0)

writer.add_mesh('my_mesh', vertices=vertices_tensor, colors=colors_tensor, faces=faces_tensor)
writer.flush()

In [13]:
for i in range(5):
    writer.add_hparams({'lr': 0.1*i, 'bsize': i},
                  {'hparam/accuracy': 10*i, 'hparam/loss': 10*i})

In [14]:
%tensorboard --host {K12AI_HOST_ADDR} --port {tb_port} --logdir {log_dir}

In [15]:
writer.close()

In [16]:
from tensorboard import notebook

In [17]:
notebook.list() # /tmp/.tensorbaord-info

Known TensorBoard instances:
  - port 8448: logdir /data/nb_data/tmp (started 1 day, 20:19:11 ago; pid 6407)
  - port 6006: logdir /data/nb_data/tmp (started 1 day, 20:25:04 ago; pid 6323)
  - port 8338: logdir /data/nb_data/tmp (started 1 day, 20:19:38 ago; pid 6395)
  - port 8748: logdir /data/nb_data/tmp (started 1 day, 20:19:03 ago; pid 6419)
  - port 9009: logdir /data/nb_data/tmp (started 1 day, 20:19:51 ago; pid 6383)
  - port 8008: logdir /data/nb_data/tmp (started 1 day, 20:21:17 ago; pid 6338)
  - port 6001: logdir /data/nb_data/tmp/mnist/logs_6001 (started 0:00:00 ago; pid 4284)


In [18]:
# notebook.display(port=6006, height=1000) # 后台已经有tensorboard进程